In [131]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import pandas as pd

import time

In [242]:
s=Service('./app/static/ressources/chromedriver.exe')

## Images won't load in the webdriver to prevent wasting resources
option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs['profile.default_content_settings'] = {'images': 2}
chrome_prefs['profile.managed_default_content_settings'] = {'images': 2}

## Variables
main_url = 'https://www.dofus.com/fr/mmorpg/communaute/ladder'
ladder_xp = {'df_name':'df_ladder_xp',
             'class_name' : 'general',
            'filter_id':'servers_222',
            'caption':'XP ladder for Ilyzaelle mono server'}
ladder_success = {'df_name':'df_ladder_success',
                  'class_name' : 'success',
            'filter_id':'servers_222',
            'caption':'Success ladder for Ilyzaelle mono server'}
ladder_pvp_solo = {'df_name':'df_ladder_pvp_solo',
                   'class_name' : 'kolizeum',
            'filter_id':'type_solo',
            'caption':'Ladder for 3v3 solo-queue PvP'}
ladder_pvp_duel = {'df_name':'df_ladder_pvp_duel',
                   'class_name' : 'kolizeum',
            'filter_id':'type_duel',
            'caption':'Ladder for 1v1 PvP'}
categories=[ladder_xp,ladder_success,ladder_pvp_solo,ladder_pvp_duel]

In [240]:
def open_ladder(browser, category):
    time.sleep(0.5)
    close_cookie = browser.find_element(By.CLASS_NAME, "ak-refuse").click()
    time.sleep(1.5)
    access_ladder = browser.find_element(By.CLASS_NAME, 'ak-section-'+category['class_name']).click()
    
def filter_ladder(browser, category):
    time.sleep(0.5)
    filter_server = browser.find_element(By.ID,category['filter_id']).click()
    time.sleep(1.5)
    
def extract_data(browser):
    cols = []
    for col in browser.find_elements(By.XPATH,"//table[contains(@class, 'ak-ladder')]/thead/tr/th"):
        cols.append(col.text)
    data = []
    num_rows = len (browser.find_elements(By.XPATH,"//table[contains(@class, 'ak-ladder')]/tbody/tr"))
    for row in range(num_rows):
        row_data=[]
        for col in browser.find_elements(By.XPATH,"//table[contains(@class, 'ak-ladder')]/tbody/tr["+str(row)+"]/td"):
            row_data.append(col.text)
        if row_data!=[]:
            data.append(row_data)
    return pd.DataFrame(data, columns=cols)

def access_data(category):
    ## Open browser in fullscreen
    browser = webdriver.Chrome(service=s, options=option)
    browser.maximize_window()
    
    ## Access main menu URL
    browser.get(main_url)
    
    open_ladder(browser, category)
    filter_ladder(browser, category)
    df = extract_data(browser)
    browser.quit()
    return df

In [244]:
def export_data():
    dfs=[]
    for category in categories :
        dfs.append(access_data(category))  
    return dfs

In [246]:
for df in export_data() :
    display(df.head())

,#,Nom,Classe,Niveau,Serveur,XP sur 24h,XP Total
0,1,Not-Specter,Osamodas,Omega 605,Ilyzaelle,0,2 072 415 695 359
1,2,Mansz,Sacrieur,Omega 596,Ilyzaelle,0,1 893 993 797 674
2,3,******,Pandawa,Omega 590,Ilyzaelle,0,1 778 423 443 050
3,4,Zolttan,Pandawa,Omega 582,Ilyzaelle,0,1 648 979 010 781
4,5,Alleria-Windrunner,Cra,Omega 581,Ilyzaelle,0,1 623 448 669 995


,#,Nom,Serveur,Classe,Niveau,Points
0,1,Pomdeupain,Ilyzaelle,Cra,Omega 250,21 219
1,1,Keity,Ilyzaelle,Huppermage,Omega 262,21 219
2,1,Paranarmol,Ilyzaelle,Sram,Omega 259,21 219
3,2,Alpha-panda,Ilyzaelle,Pandawa,Omega 200,21 149
4,2,Halkave,Ilyzaelle,Xelor,Omega 177,21 149


,#,Nom,Serveur,Classe,Niveau,Points,Cote,Victoires
0,1,Belle-Cra,Atcham,Cra,Omega 123,8614,3195,203
1,2,Vodka-Cerize-[Ota],Brumen,Cra,Omega 131,6559,3372,350
2,3,Kadi-I,Nidas,Cra,Omega 73,6435,3336,168
3,4,Moneys-money,Pandore,Cra,Omega 26,6075,2901,141
4,5,Draconis-Aurum,Julith,Sacrieur,87,5730,1589,269


,#,Nom,Serveur,Classe,Niveau,Cote
0,1,Minkow,Merkator,Sadida,198,5275
1,2,Ultrinum,Nidas,Sadida,198,5255
2,3,Tsr-creww,Furye,Steamer,198,5087
3,4,Klump,Ush,Steamer,199,5049
4,5,Phoque-lapeaulisse,Pandore,Ecaflip,194,5047
